#* 학습 데이터셋 만들기위한 레이블링
- 분별 레이블
- 일별 레이블

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
# !pip install JPype1
# !pip install konlpy
# !pip install pyMySQL
# !pip install -U finance-datareader

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
# 리스트 flatten을 위한 itertools 패키지 import
import itertools
import warnings
warnings.filterwarnings('ignore')

1. 클러스터 별로 종목번호 읽어서
3. 변동률(분당) 구해서 넣고
4. P/N/H 레이블 붙이기
5. 뉴스 속보에 넣을 레이블 따로 DB 저장
- 상승 U
- 하락 D
- 보합 H

In [ ]:
# --get_one_stock_price()-----------------------------------------------
# 종목별 분봉 데이터 가져오기
def get_one_stock_price(stock_code):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="비밀번호", db="DB이름", charset = 'utf8')
  # 종목 지정:  stock_code
  scode = stock_code
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM DB이름.A{scode} x"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  print(scode)
  # DB close
  conn.close()
  # 데이터 리턴
  return result_df
# --get_one_stock_price() :END----------------------------------------

# --get_cluster_stock_code()------------------------------------------
# 한 클러스터내 종목들 읽어오기
def get_cluster_stock_code(cluster_number):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 클러스터 지정:  cluster_number
  cnumber = cluster_number
  # 실행확인을 위한 화면 출력
  print(cnumber)
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM cluster_{cnumber} x"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # DB close
  conn.close()
  # 데이터 리턴
  return result_df
# --get_cluster_stock_code()------------------------------------------

# --get_min_change_rate_label()---------------------------------------
# 분당 변동율(%) 구하기
def get_min_change_rate_label(df_stock):
  # 원본 copy 만들기
  df_min= df_stock.copy()
  # 분당 변동율(%) 레이블 값 넣기
  df_min['bun_rate'] = ((df_stock['close']-df_stock['open'])/df_stock['open'])*100
  # 분당 변동율(%) 레이블 넣기: Up,Hold,Down
  df_min.loc[df_min['bun_rate']>0,'bun_label'] = 'U'
  df_min.loc[df_min['bun_rate']==0,'bun_label'] = 'H'
  df_min.loc[df_min['bun_rate']<0,'bun_label'] = 'D'
  # 실행 확인을 위한 화면 출력
  print(df_min.head())
  # df 리턴
  return df_min
# --get_min_change_rate_label() :END --------------------------------

# --df_labelDBsave()-------------------------------------------------
# 레이블 된 종목 DB 저장
def df_labelDBsave(df_stock, stock_code):
  # DB 커넥션 생성
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
  df_stock.to_sql(name='A'+stock_code+'_bun_rate_label', con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()
  print(stock_code)
# --df_labelDBsave() :END-------------------------------------------
# --save_bun_rate_label_by_cluster()--------------------------------
# 클러스터별로 분당 변동률과 레이블 구해서 따로 저장하기
# 클러스터 별 종목 코드 가져오기
def save_bun_rate_label_by_cluster(cluster_number):
  # 실행 확인용 화면 출력
  print(cluster_number)
  # 클러스터의 종목 코드 리스트 
  stock_code_list = get_cluster_stock_code(cluster_number)['code']
  # 모든 종목에 대해 적용
  for stock_code in stock_code_list:
    # 실행 확인용 화면 출력
    print('-',stock_code)
    # 종목의 OHLCV 데이터 가져오기
    df_stock = get_one_stock_price(stock_code)
    # 분당 변동률과 레이블 더하기
    df_stock = get_min_change_rate_label(df_stock)
    # 종목별 변동률과 레이블만 DB 테이블로 저장: 예'A005930_min_rate_label'
    df_stock = df_stock[['date','bun_rate','bun_label']]
    # 실행 확인용 화면 출력
    print('--',df_stock.head(3))
    # DB에 저장
    df_labelDBsave(df_stock,stock_code)
# --save_bun_rate_label_by_cluster() :END----------------------------


In [ ]:
###################################################################
# 메인 함수 실행
# 클러스별로 분별 레이블 달기
save_bun_rate_label_by_cluster(4)
###################################################################

---
* 수집한 종목과 클러스터 분류된 종목간 검증하기
- 수집한 종목 아닌 것들은 삭제하기

In [ ]:
# 수집한 종목과 클러스터 종목 비교
# --get_stock_code_list()-----------------------------------------------
# 수집한 종목 코드 리스트 가져오기
def get_stock_code_list():
  # DB 연결 설정
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 테이블명 읽어오기 위한 sql 준비
  sql = f"SELECT table_name FROM information_schema.tables WHERE table_schema = 'DB이름'"
  # DB 검색결과를 dataframe에 저장
  df_table_names = pd.read_sql_query(sql, conn)
  # DB 연결 close
  conn.close()
  # 종목코드에서 첫글자'A' 제거
  result_code=df_table_names[df_table_names['table_name'].str.startswith('A')]['table_name'].str[1:]
  # 종목리스트 반환
  return result_code
# --get_stock_code_list() :END------------------------------------------

# --get_cluster_stock_code()------------------------------------------
# 한 클러스터내 종목들 읽어오기
def get_cluster_stock_code(cluster_number):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 클러스터 지정:  cluster_number
  cnumber = cluster_number
  # 실행확인을 위한 화면 출력
  print(cnumber)
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM cluster_{cnumber} x"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # DB close
  conn.close()
  # 데이터 리턴
  return result_df
# --get_cluster_stock_code()------------------------------------------


In [ ]:
#save_bun_rate_label_by_cluster(8)
#save_bun_rate_label_by_cluster(7)
#save_bun_rate_label_by_cluster(6)
#save_bun_rate_label_by_cluster(5)
save_bun_rate_label_by_cluster(4)
#save_bun_rate_label_by_cluster(3)
#save_bun_rate_label_by_cluster(2)
#save_bun_rate_label_by_cluster(1)
#save_bun_rate_label_by_cluster(0)